In [0]:
%matplotlib inline

import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold, train_test_split

warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 40)
random_state = 123456789

In [0]:
URL = 'https://raw.githubusercontent.com/ccny-data/College_Major_Data/master/data/IPEDS_data%20-%20Data.csv'
df = pd.read_csv(URL)

In [97]:
df.shape

(1534, 145)

In [98]:
df.sample(5)

,ID number,Name,year,ZIP code,Highest degree offered,County name,Longitude location of institution,Latitude location of institution,Religious affiliation,Offers Less than one year certificate,Offers One but less than two years certificate,Offers Associate's degree,Offers Two but less than 4 years certificate,Offers Bachelor's degree,Offers Postbaccalaureate certificate,Offers Master's degree,Offers Post-master's certificate,Offers Doctor's degree - research/scholarship,Offers Doctor's degree - professional practice,Offers Doctor's degree - other,...,Percent of first-time undergraduates - out-of-state,Number of first-time undergraduates - foreign countries,Percent of first-time undergraduates - foreign countries,Number of first-time undergraduates - residence unknown,Percent of first-time undergraduates - residence unknown,"Graduation rate - Bachelor degree within 4 years, total","Graduation rate - Bachelor degree within 5 years, total","Graduation rate - Bachelor degree within 6 years, total",Percent of freshmen receiving any financial aid,"Percent of freshmen receiving federal, state, local or institutional grant aid",Percent of freshmen receiving federal grant aid,Percent of freshmen receiving Pell grants,Percent of freshmen receiving other federal grant aid,Percent of freshmen receiving state/local grant aid,Percent of freshmen receiving institutional grant aid,Percent of freshmen receiving student loan aid,Percent of freshmen receiving federal student loans,Percent of freshmen receiving other loan aid,Endowment assets (year end) per FTE enrollment (GASB),Endowment assets (year end) per FTE enrollment (FASB)
1483,366711,California State University-San Marcos,2013,92096-0001,Master's degree,San Diego County,-117.158013,33.129115,Not applicable,Implied no,Implied no,Implied no,Implied no,Yes,Implied no,Yes,Implied no,Implied no,Implied no,Implied no,...,1.0,22.0,1.0,0.0,0.0,13.0,37.0,48.0,73.0,57.0,45.0,45.0,7.0,37.0,44.0,45.0,44.0,3.0,2284.0,NaN
1139,214883,Peirce College,2013,19102-4699,Master's degree,Philadelphia County,-75.166332,39.945483,Not applicable,Implied no,Yes,Yes,Implied no,Yes,Implied no,Yes,Implied no,Implied no,Implied no,Implied no,...,6.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,87.0,87.0,65.0,65.0,35.0,22.0,35.0,74.0,74.0,4.0,NaN,164.0
884,196200,SUNY College at Potsdam,2013,13676-2294,Master's degree,St. Lawrence County,-74.976889,44.662862,Not applicable,Implied no,Implied no,Implied no,Implied no,Yes,Implied no,Yes,Implied no,Implied no,Implied no,Implied no,...,3.0,6.0,1.0,0.0,0.0,30.0,49.0,51.0,91.0,77.0,47.0,46.0,12.0,59.0,73.0,77.0,77.0,6.0,4926.0,NaN
716,181020,Doane College-Crete,2013,68333,Bachelor's degree,Saline County,-96.949656,40.623107,Not applicable,Implied no,Implied no,Implied no,Implied no,Yes,Implied no,Implied no,Implied no,Implied no,Implied no,Implied no,...,26.0,3.0,1.0,0.0,0.0,51.0,60.0,60.0,100.0,100.0,35.0,34.0,19.0,18.0,100.0,93.0,93.0,8.0,NaN,91376.0
324,148335,Robert Morris University Illinois,2013,60605,Master's degree,Cook County,-87.627068,41.876278,Not applicable,Implied no,Implied no,Yes,Implied no,Yes,Implied no,Yes,Implied no,Implied no,Implied no,Implied no,...,8.0,5.0,1.0,0.0,0.0,76.0,78.0,80.0,97.0,94.0,64.0,64.0,21.0,43.0,80.0,81.0,81.0,3.0,NaN,13240.0


In [0]:
df = df.dropna(subset=['Admissions total', 'Applicants total'])

In [0]:
df['is_religious'] = (df['Religious affiliation'] != 'Not applicable')*1
df['offers_associates'] = (df["Offers Associate's degree"] == 'Yes')*1
df['offers_bachelors'] = (df["Offers Bachelor's degree"] == 'Yes')*1
df['offers_masters'] = (df["Offers Master's degree"] == 'Yes')*1
df['is_private'] = (df['Control of institution'] == 'Private not-for-profit')*1
df['is_public'] = (df['Control of institution'] == 'Public')*1
#df['is_urban'] = (df['Degree of urbanization (Urban-centric locale)']=='City: Large')*1
#df['is_suburb'] = ('Suburb' in str(df['Degree of urbanization (Urban-centric locale)']))*1
#df['is_town'] = ('Town' in str(df['Degree of urbanization (Urban-centric locale)']))*1
#df['is_rural'] = ('Rural' in str(df['Degree of urbanization (Urban-centric locale)']))*1

In [101]:
df['Degree of urbanization (Urban-centric locale)'].value_counts()

City: Large        269
Suburb: Large      259
City: Small        199
City: Midsize      167
Town: Distant      148
Town: Remote       116
Town: Fringe        56
Rural: Fringe       47
Suburb: Midsize     47
Suburb: Small       31
Rural: Distant      25
Rural: Remote       13
Name: Degree of urbanization (Urban-centric locale), dtype: int64

In [102]:
df.sample(3)

,ID number,Name,year,ZIP code,Highest degree offered,County name,Longitude location of institution,Latitude location of institution,Religious affiliation,Offers Less than one year certificate,Offers One but less than two years certificate,Offers Associate's degree,Offers Two but less than 4 years certificate,Offers Bachelor's degree,Offers Postbaccalaureate certificate,Offers Master's degree,Offers Post-master's certificate,Offers Doctor's degree - research/scholarship,Offers Doctor's degree - professional practice,Offers Doctor's degree - other,...,"Graduation rate - Bachelor degree within 5 years, total","Graduation rate - Bachelor degree within 6 years, total",Percent of freshmen receiving any financial aid,"Percent of freshmen receiving federal, state, local or institutional grant aid",Percent of freshmen receiving federal grant aid,Percent of freshmen receiving Pell grants,Percent of freshmen receiving other federal grant aid,Percent of freshmen receiving state/local grant aid,Percent of freshmen receiving institutional grant aid,Percent of freshmen receiving student loan aid,Percent of freshmen receiving federal student loans,Percent of freshmen receiving other loan aid,Endowment assets (year end) per FTE enrollment (GASB),Endowment assets (year end) per FTE enrollment (FASB),is_religious,offers_associates,offers_bachelors,offers_masters,is_private,is_public
1208,218742,University of South Carolina-Upstate,2013,29303-4999,Master's degree,Spartanburg County,-81.970583,34.996403,Not applicable,Implied no,Implied no,Implied no,Implied no,Yes,Yes,Yes,Implied no,Implied no,Implied no,Implied no,...,37.0,40.0,97.0,94.0,50.0,50.0,17.0,83.0,34.0,70.0,69.0,5.0,1349.0,NaN,0,0,1,1,0,1
1204,218654,University of South Carolina-Beaufort,2013,29909,Bachelor's degree,Beaufort County,-80.973595,32.303308,Not applicable,Implied no,Implied no,Yes,Implied no,Yes,Implied no,Implied no,Implied no,Implied no,Implied no,Implied no,...,22.0,27.0,91.0,81.0,42.0,42.0,6.0,57.0,29.0,72.0,71.0,4.0,50.0,NaN,0,1,1,0,0,1
137,123961,University of Southern California,2013,90089,Doctor's degree - research/scholarship and pro...,Los Angeles County,-118.283858,34.021058,Not applicable,Implied no,Implied no,Implied no,Implied no,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,90.0,91.0,67.0,59.0,18.0,18.0,17.0,12.0,59.0,34.0,34.0,2.0,NaN,107715.0,0,0,1,1,1,0


In [0]:
# Lots of features here, might be worth looking into finding which are most importand and trim the least important
# Note: the Total enrollment column, it has two spaces between "Total" and "enrollment".

features = ['is_religious', 'offers_associates', 'offers_bachelors', 'offers_masters', 'Percent of freshmen submitting SAT scores', 'Percent of freshmen submitting ACT scores', 'SAT Critical Reading 25th percentile score', 
            'SAT Critical Reading 75th percentile score', 'SAT Math 25th percentile score', 'SAT Math 75th percentile score', 'SAT Writing 25th percentile score', 'SAT Writing 75th percentile score', 
            'ACT Composite 25th percentile score', 'ACT Composite 75th percentile score', 'Tuition and fees, 2013-14', 'Total  enrollment', 'Percent of total enrollment that are American Indian or Alaska Native',
           'Percent of total enrollment that are Asian', 'Percent of total enrollment that are Black or African American', 'Percent of total enrollment that are Hispanic/Latino', 
            'Percent of total enrollment that are Native Hawaiian or Other Pacific Islander', 'Percent of total enrollment that are White', 'Percent of total enrollment that are two or more races', 'Percent of total enrollment that are women',
           'Percent of freshmen receiving any financial aid', 'Graduation rate - Bachelor degree within 4 years, total', 'Graduation rate - Bachelor degree within 6 years, total']

In [0]:
model_df = df[(features + ['Percent admitted - total'])].dropna().reset_index()

train_df, holdout_df, y_train, y_holdout = train_test_split(
    model_df[features], 
    model_df['Percent admitted - total'], test_size=0.1,
    random_state=random_state)

train_df['Percent admitted - total'] = y_train
holdout_df['Percent admitted - total'] = y_holdout

train_df.reset_index(inplace=True)
holdout_df.reset_index(inplace=True)


In [0]:
k_fold = KFold(n_splits=5, random_state=random_state)

In [0]:
def get_cv_results(classifier):
    
    results = []
    for train, test in k_fold.split(train_df):
        classifier.fit(train_df.loc[train, features], train_df.loc[train, 'Percent admitted - total'])
        y_predicted = classifier.predict(train_df.loc[test, features])
        accuracy = accuracy_score(train_df.loc[test, 'Percent admitted - total'], y_predicted)
        results.append(accuracy)
        #print(accuracy)
    
    return np.mean(results), np.std(results)

In [107]:
logreg = LogisticRegression(
    random_state=random_state, 
    solver='lbfgs'
)

get_cv_results(logreg)

(0.02637853949329359, 0.015082951538233054)

In [108]:
dtree = DecisionTreeRegressor(
    random_state=random_state, 
    #max_depth=max_depth
)

get_cv_results(dtree)

(0.026337894594228427, 0.014168853010837902)

In [110]:
rforest = RandomForestRegressor(
    random_state=random_state, 
    #max_depth=max_depth,
    n_estimators=100
)

get_cv_results(rforest)

ValueError: ignored

In [111]:
gbm = GradientBoostingRegressor(
    random_state=random_state, 
    #max_depth=max_depth,
    n_estimators=100
)

get_cv_results(gbm)

ValueError: ignored